# Decision trees and forests

<img src="https://imgs.xkcd.com/comics/flow_charts.png">

Decision trees can be used in both classification and regression problems and are a class of supervised learning techniques. Thet operate by recursively dividing the feature space into adjacent cubiod regions by learning a hierarchy of simple binary models each working on a single variable in a tree structure. They are relatively simple models which work well on both numerical and categorical variables and are strongly favoured for their interpretability.

eg

It’s important to note that each split is based on a single variable. If boundaries more complicated than a simple inequality are used, it becomes unfeasibly difficult to generate trees at all. Still, given the combinatorial complexity, trees are optimised by greedy means, starting with a single root node, corresponding to the whole input space, and then growing the tree by adding nodes one at a time incorporating empirical heuristics. Decision trees are built by recursively adding nodes to a binary tree, with leaf nodes corresponding to output variables or classes, to minimise a loss function such as SSE or MSE. In practice, it would be relatively trivial to minimise such as loss function by splitting the input space into N subregions for an N sized training set, with each leaf of the tree containing a single training point. Therefore, it is important to consider when to stop adding new nodes. It is common practice to grow a large tree, using a stopping criterion based on the number
of data points associated with the leaf nodes, and then prune back the resulting tree based on redundent or minimally informative nodes.

There are 2 main algorithms commonly used in a variety of domains which are effective in constructing both decision and classification trees. These are CART (Classification And Regression Tree) and C4.5

# CART

A tree is grown starting from the root node by repeatedly using the following steps on each leaf node.
1. Find the best split for each variable
2. Among the best splits found in step 1, choose the one that maximises or minimises the splitting criterion.
3. Split the node using its best split found in step 2 if the stopping rules are not satisfied.


### Stopping criteria:
Nodes will not be split if:
* If all examples in a node have identical values/class.
* If all examples in a node have identical values for each predictor variable.
* If the size of a node is less than a predefined minimum node size.
* If the split of a node results in a child node whose size is less than the specified minimum.
* If the depth of the node reaches a predefined limit.
* If the improvement in the loss metric is below some threshold.

Splites are made based on information gain. The most popular metrics for information gain are entropy and the Gini impurity measure.

If there are C total classes and p(i) is the probability of picking a datapoint with class i, then the Gini Impurity is calculated as:
$$\sum^C_{i=1}p(i)(1-p(i))$$

In [1]:
import numpy as np
from sklearn.datasets import load_iris

In [2]:
data = load_iris()

In [3]:
def gini_impurity(classes):
    assert isinstance(classes, (np.ndarray, list))
    if type(classes) == list:
        classes = np.array(classes)
        
    impurity = 1
    for i in set(classes):
        p = np.mean(classes == i)
        impurity -= p**2
    return impurity

In [4]:
def gini_impurity2(classes):
    assert isinstance(classes, (np.ndarray, list))
    if type(classes) == list:
        classes = np.array(classes)
        
    impurity = 0
    for i in set(classes):
        p = np.mean(classes == i)
        impurity += p*(1-p)
    return impurity

In [5]:
def split(data, col, val, classes=None):
    idx = data[..., col] <= val
    l = data[idx]
    r = data[~idx]
    if classes is not None:
        lc = classes[idx]
        rc = classes[~idx]
        return (l, lc), (r, rc)
    return l, r

In [6]:
class Node():
    def __init__(self):
        self.leaf = True
        
        self.split_column = None
        self.split_value = None
        self.value = None
        
        self.l_branch = None
        self.r_branch = None
        
    def __call__(self, data):
        if self.leaf:
            return self.value
        elif data[self.split_column] <= self.split_value:
            return self.l_branch(data)
        else:
            return self.r_branch(data)

In [7]:
def build_tree(inputs, targets, root=None):
    
    if root is None:
        root = Node()
    
    #evaluate splits
    minpurity = 1
    mc = mv = -1
    for col in range(inputs.shape[-1]):
        for val in set(inputs[:, col]):
            (lsplit, lclasses), (rsplit, rclasses) = split(inputs, col, val, targets)
            impurity = gini_impurity(lclasses) + gini_impurity(rclasses)
            if impurity < minpurity:
                minpurity = impurity
                mc = col
                mv = val
    if len(lclasses) == 0 or len(rclasses) == 0 or mc == -1:
        return
    
    root.split_column = mc
    root.split_value = mv
    root.leaf = False
        
    root.l_branch = Node()
    build_tree(lsplit, lclasses, root.l_branch)
    
    root.r_branch = Node()
    build_tree(rsplit, rclasses, root.r_branch)
    
    return root

For regression, we used the only change needed is to the to loss function. Usually the squared-error is used.

# C4.5

# Random Forest

In [8]:
# in progress